# **GluonCV Demo: Instance Segmentation using Mask R-CNN**

In [0]:
# 1. Install MXNet and GluonCV, load Python Packages

! pip install -q mxnet gluoncv --progress-bar off
from gluoncv import model_zoo, data, utils
from matplotlib import pyplot as plt

# 2. Upload an Image File
# We can upload multiple image files at the same time, but try upload one image file only.

from google.colab import files
uploaded = files.upload()

# 3. Load Mask R-CNN Pre-trained Model
# We use Mask R-CNN model trained on Microsoft COCO dataset
# Other available models: https://gluon-cv.mxnet.io/model_zoo/segmentation.html

MaskRCNN = model_zoo.get_model('mask_rcnn_resnet50_v1b_coco', pretrained=True)

# 4. Print out the Class Names supported by Mask R-CNN
print('Total number of classes recognized by Mask R-CNN:', len(MaskRCNN.classes))
print('The classes are:', MaskRCNN.classes)

# 5. Transform the Image File to Suitable Format
# At step 2 we can upload multiple image files at the same time, but here we only use the first image file.
# Transformation default: short side = 600 pixels, long side <= 1000 pixels, keep aspect ratio.
# new_data: Numerical data that will be passed to Faster R-CNN for object detection
# new_image: Newly transformed image according to Faster R-CNN requirement

new_data, new_image = data.transforms.presets.rcnn.load_test(list(uploaded.keys())[0])

# 6. Apply Mask R-CNN to do Instance Segmentation
# box_ids: The class ID of bounding box (which kind of object is detected)
# scores: The probability that the bounding box contains the detected object
# bboxes: Bounding box coordinates
# masks: Masks showing which pixel corresponds to which class

box_ids, scores, bboxes, masks = MaskRCNN(new_data)

# 7. Display the Instance Segmentation Result
# Expand instance segmentation mask to full image size
width, height = new_image.shape[1], new_image.shape[0]
new_masks, _ = utils.viz.expand_mask(masks[0], bboxes[0], (width, height), scores[0])
new_image = utils.viz.plot_mask(new_image, new_masks)
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1)

# Threshold = 0.5 means we only display those bounding boxes with scores >= 0.5
ax = utils.viz.plot_bbox(new_image, bboxes[0], scores[0], box_ids[0], class_names=MaskRCNN.classes, thresh=0.5, ax=ax)
plt.show()

# 8. Print out the Numerical Result of Instance Segmentation
for i in range(len(box_ids[0])):
    if box_ids[0][i] != -1 and scores[0][i].asscalar() >= 0.5:
        print('Class =', MaskRCNN.classes[int(box_ids[0][i].asscalar())], ' Bounding Box =', bboxes[0][i].asnumpy(), ' Score =', scores[0][i].asscalar())